In [1]:
import datetime as dt
import numpy    as np
import os.path
import pandas   as pd
import random   as rand
import seaborn  as sns
from pytz import *

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
fulldata2015_filepath = '../../data/raw/wroclawski-rower-miejski/wypozyczenia_wrm-sezon2015.xlsx'
sample2015_filepath   = '../../data/raw/wroclawski-rower-miejski/wypozyczenia_wrm-sezon2015_sample.xlsx'

fulldata2016_filepath = '../../data/raw/wroclawski-rower-miejski/wypozyczenia_wrm-sezon2016.xlsx'
sample2016_filepath   = '../../data/raw/wroclawski-rower-miejski/wypozyczenia_wrm-sezon2016_sample.xlsx'

use_sample = True

## Bike rentals

In [3]:
def create_sample(fulldata_filepath, sample_filepath, number_of_samples=98):
    """
    Sample will contain first and last record and other random records.
    """
    df = pd.read_excel(fulldata_filepath)
    sample_indices = [0, df.index[-1]] + rand.sample(range(1, df.index[-2]), number_of_samples)
    sample_indices.sort()
    sample_df = df.iloc[sample_indices, :]
    sample_df.to_excel(sample_filepath, index=False)

if use_sample:
    if not os.path.exists(sample2015_filepath):
        create_sample(fulldata2015_filepath, sample2015_filepath)

    if not os.path.exists(sample2016_filepath):
        create_sample(fulldata2016_filepath, sample2016_filepath)
        
    rentals2015_df = pd.read_excel(sample2015_filepath)      
    rentals2016_df = pd.read_excel(sample2016_filepath)    
else:
    rentals2015_df = pd.read_excel(fulldata2015_filepath)
    rentals2016_df = pd.read_excel(fulldata2016_filepath)

In [4]:
rentals = pd.concat([rentals2015_df, rentals2016_df])

#### Feature engineering

In [5]:
rentals['Godzina wynajmu'] = rentals['Data wynajmu'].apply(lambda x: dt.datetime(x.year, x.month, x.day, x.hour, 0))

In [6]:
rentals_per_hour = pd.DataFrame(rentals.groupby('Godzina wynajmu').count()['Numer roweru']).reset_index()
rentals_per_hour.columns = ['Godzina wynajmu', 'Ilość wypożyczeń']

rentals = rentals.join(rentals_per_hour.set_index('Godzina wynajmu'), on='Godzina wynajmu', how='left')

In [7]:
def get_season_start_date_and_end_date(df):
    index_first = 0
    index_last  = df.index[-1]

    start_date = df.loc[index_first, "Data wynajmu"].date()
    end_date   = df.loc[index_last, "Data zwrotu"].date()
    return start_date, end_date

rentals2015_start_date, rentals2015_end_date = get_season_start_date_and_end_date(rentals2015_df)
print('Season 2015 started on: ', rentals2015_start_date)
print('Season 2015 ended on:   ', rentals2015_end_date)

rentals2016_start_date, rentals2016_end_date = get_season_start_date_and_end_date(rentals2016_df)
print('Season 2016 started on: ', rentals2016_start_date)
print('Season 2016 ended on:   ', rentals2016_end_date)

Season 2015 started on:  2015-04-28
Season 2015 ended on:    2015-11-30
Season 2016 started on:  2016-03-01
Season 2016 ended on:    2016-11-30


In [8]:
# Our final dataset will have rentals data aggregated 'per hour'
year2015 = pd.date_range(rentals2015_start_date, rentals2015_end_date + dt.timedelta(days=1), freq='H')
year2016 = pd.date_range(rentals2016_start_date, rentals2016_end_date + dt.timedelta(days=1), freq='H')
dates_frame = pd.DataFrame({'date': year2015.append(year2016)})

## Weather dataset

In [ ]:
# load data
weather_df = pd.read_csv('../../data/raw/wroclawski-rower-miejski/dane_pogoda.csv', sep=';')

In [ ]:
# TODO: fix problem with changing time zones from UTC+2 to UTC+1 and vice versa
weather_df['correct_pl_date'] = \
    pd.to_datetime(weather_df['dt_iso'], format="%Y-%m-%d %H:%M:%S +0000 UTC") + pd.DateOffset(hours=2)

In [ ]:
weather_df.head()

In [ ]:
weather_dataset = pd.read_csv('../../data/raw/wroclawski-rower-miejski/dane_pogoda.csv', sep=';')

In [ ]:
weather_dataset.head()

In [ ]:
weather_dataset['dt_iso'].head()

In [ ]:
year2015

In [ ]:
weather_dataset['correct_pl_date'] = pd.to_datetime(weather_dataset['dt_iso'], format="%Y-%m-%d %H:%M:%S +0000 UTC") + pd.DateOffset(hours=2)

In [ ]:
weather_dataset['correct_pl_date']

In [ ]:
weather_dataset.drop_duplicates(subset=['correct_pl_date'], inplace=True)

In [ ]:
merged_dataset = pd.merge(left=dates_frame, right=weather_dataset, left_on='date', right_on='correct_pl_date', how='left')

In [ ]:
merged_dataset.isnull().sum()

In [ ]:
col_number = merged_dataset.shape[1]

merged_dataset['is_na_filled'] = 0
for index, row in merged_dataset.iterrows():
    if pd.isnull(row['dt']):
        merged_dataset.iloc[index, 1:col_number] = merged_dataset.iloc[index - 1, 1:col_number]
        merged_dataset.loc[merged_dataset.index[index], 'is_na_filled'] = 1

In [ ]:
columns_with_na = list(merged_dataset.loc[:, merged_dataset.isnull().sum() > 0].columns)

In [ ]:
merged_dataset.drop(columns_with_na + ['weather_icon', 'dt', 'dt_iso', 'correct_pl_date', 'city_id', 'weather_id'], axis=1, inplace=True)

In [ ]:
merged_dataset

## Bike rentals

In [ ]:
#rentals2015 = pd.read_csv('wypozyczenia_wrm-sezon2015.csv', encoding = "ISO-8859-1", sep=';')
#rentals2016 = pd.read_csv('wypozyczenia_WRM-sezon2016.csv', encoding = "ISO-8859-1", sep=';')
#rentals = pd.concat([rentals2015_df, rentals2016_df])

#rentals['rental_date'] = pd.to_datetime(rentals['Data wynajmu'], format='%d.%m.%Y %H:%M')

#rentals['rental_hour'] = rentals['rental_date'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, dt.hour, 0))

#rentals_per_hour = pd.DataFrame(rentals.groupby('rental_hour').count()['Numer roweru']).reset_index()
#rentals_per_hour.columns = ['rental_hour', 'rentals_no']

In [ ]:
all_data = pd.merge(left=merged_dataset, right=rentals_per_hour, left_on='date', right_on='rental_hour', how='left')

In [ ]:
all_data.loc[all_data['rentals_no'].isnull(),'rentals_no'] = 0

In [ ]:
all_data.drop('rental_hour', axis=1, inplace=True)

In [ ]:
all_data.shape

In [ ]:
len(all_data['date'].unique())

In [ ]:
all_data['rentals_no'].hist()

In [ ]:
all_data['temp'] = all_data['temp'] - 273.15
all_data['temp_min'] = all_data['temp_min'] - 273.15
all_data['temp_max'] = all_data['temp_max'] - 273.15

In [ ]:
all_data['is_weekend'] = all_data['date'].dt.weekday.isin([5,6])

In [ ]:
# days without work
free_days = ['2016-01-01', '2016-01-06', '2016-03-27', '2016-03-28', '2016-05-01', '2016-05-03', '2016-05-15', '2016-05-26', '2016-08-15', '2016-11-01', '2016-11-11', '2016-12-25', '2016-12-26',
'2015-01-01', '2015-01-06', '2015-04-05', '2015-04-06', '2015-05-01', '2015-05-03', '2015-05-24', '2015-06-04', '2015-08-15', '2015-11-01', '2015-11-11', '2015-12-25', '2015-12-26']

free_days_dt = [pd.Timestamp(x) for x in free_days]

In [ ]:
all_data['bank_holidays'] = all_data.date.apply(lambda date: datetime.datetime(date.year, date.month, date.day) in free_days_dt)


In [ ]:
all_data['is_not_working_day'] = all_data['bank_holidays'] | all_data['is_weekend']

In [ ]:
all_data['month'] = all_data['date'].apply(lambda x: x.strftime('%b'))

In [ ]:
test = all_data[all_data['date'] >= '2016-08-15']
print(test.shape)

In [ ]:
train = all_data[all_data['date'] < '2016-08-15']
print(train.shape)

In [ ]:
test.drop('rentals_no', axis=1, inplace=True)

In [ ]:
print(test.shape)

In [ ]:
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

In [ ]:
print(train.columns)

In [ ]:
train